In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import skimage as sk
import random 


IMG = sk.io.imread('Star.png', as_gray=True)
IMG = sk.transform.resize(IMG, (500, 500))
IMG = IMG > 0
IMG = IMG.astype(np.uint8) * 255
rr,cc = sk.draw.circle_perimeter(IMG.shape[0] // 2, IMG.shape[1] // 2, IMG.shape[0] // 2 -1)
IMG[rr, cc] = 128

def countTheBlack(image):
    return np.sum(image == 0)

FullBlackCount = countTheBlack(IMG)
print("FullBlackCount: ", FullBlackCount)

In [ ]:
def getCircleCodrs(num):
    radians = np.radians(num)
    x = int((np.cos(radians) * IMG.shape[1] / 2 + IMG.shape[1] / 2)) -1
    y = int((np.sin(radians) * IMG.shape[0] / 2+ IMG.shape[0] / 2)) -1
    return x, y

def drawALine(num1, num2, image):
    x1, y1 = getCircleCodrs(num1)
    x2, y2 = getCircleCodrs(num2)
    rr, cc = sk.draw.line(x1, y1, x2, y2)
    image[rr, cc] = 128

In [ ]:
POPULATION_SIZE = 100
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.9
GENLEN = 50
ExpectedSuccessRate = 0.8
ExpectedSuccessRate = 1 - ExpectedSuccessRate

In [ ]:
class Individual(object): 
	def __init__(self, chromosome): 
		self.chromosome = chromosome 
		self.fitness = self.cal_fitness() 

	@classmethod
	def mutated_genes(self): 
		gene = random.randint(0, 360)
		return gene 

	@classmethod
	def create_gnome(self):
		return [self.mutated_genes() for _ in range(GENLEN)] 

	def mate(self, par2): 
		global MUTATION_RATE
		child_chromosome = [] 
		for gp1, gp2 in zip(self.chromosome, par2.chromosome):	 
			prob = random.random() 
			parRate = (1 - MUTATION_RATE) / 2
			if prob < parRate: 
				child_chromosome.append(gp1)
			elif prob < parRate * 2:
				child_chromosome.append(gp2)
			else:
				child_chromosome.append(self.mutated_genes())

		return Individual(child_chromosome) 
	
	def cal_fitness(self): 
		global IMG
		image = np.copy(IMG)
		i = 0
		for j in range(1,len(self.chromosome)):
			drawALine(self.chromosome[i], self.chromosome[j], image)
			i = j
		return countTheBlack(image)

	def drawTheImage(self):
		global IMG
		# image = np.copy(IMG)
		image = np.zeros(IMG.shape)
		i = 0
		for j in range(1,len(self.chromosome)):
			drawALine(self.chromosome[i], self.chromosome[j], image)
			i = j
		plt.imshow(image, cmap='gray')
		plt.show()

In [ ]:
def main(): 
	global POPULATION_SIZE,CROSSOVER_RATE,ExpectedSuccessRate,FullBlackCount

	generation = 1

	found = False
	population = [] 
	pastFitness = 0
	for _ in range(POPULATION_SIZE): 
				gnome = Individual.create_gnome() 
				population.append(Individual(gnome)) 

	while not found: 
		population = sorted(population, key = lambda x:x.fitness) 

		if population[0].fitness / FullBlackCount <= ExpectedSuccessRate: 
			found = True
			break

		new_generation = [] 

		s = int((1 - CROSSOVER_RATE) * POPULATION_SIZE) 
		new_generation.extend(population[:s]) 

		s = int(CROSSOVER_RATE * POPULATION_SIZE) 
		for _ in range(s): 
			parent1 = random.choice(population[:50]) 
			parent2 = random.choice(population[:50]) 
			child = parent1.mate(parent2) 
			new_generation.append(child) 

		population = new_generation 
		if (population[0].fitness != pastFitness):
			pastFitness = population[0].fitness
			print("Generation: {}\tFitness: {}".\
				format(generation,
				population[0].fitness)) 
			population[0].drawTheImage()

		generation += 1

	
	print("Generation: {}\tFitness: {}".\
		format(generation, 
		population[0].fitness)) 
	population[0].drawTheImage()

if __name__ == '__main__': 
	main() 